# Semantic Search using Language Models and Nearest Neighbor Indexes

Go ahead and install these libraries


In [20]:
 ! pip install transformers ir-measures torch 

  Using cached pyserini-0.18.0-py3-none-any.whl (114.8 MB)
  Using cached onnxruntime-1.12.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.5 MB)


  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.6.0
    Uninstalling onnxruntime-1.6.0:
      Successfully uninstalled onnxruntime-1.6.0


Cross encoders are an easy to understand application of Language Models as they treate relevance as a binary classification task. A query and a document are concatenated and separated by [SEP] token and a model learns to predict if a query document pair is relevant or not. This method is incredibly effective despite its simplicity. While effective, this approach is not efficient from an inference perspective. If we want to run this on 10,000 documents and 10,000 queries we will have to use our model 100m times while the same approach requires 20,000 model calls for a bi-encoder. This inefficiency is why cross-encoders are mostly used for reranking. 

Before we explore cross encoders for reranking and relevance estimation. We will be using the Sentence Transformers library which is a Language Model library which is optimized for representing sentences as text.

## ReRanking With Transformers

Import the needed libraries

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

Load the model and the tokenizer, which prepares text for usage with the model

In [3]:
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-electra-base')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-electra-base')

Lets go ahead and look at some examplex. Note that for each comparison I am repeating the query. Each usage requires another complete run of the model

In [ ]:
features = tokenizer(['How many people live in Berlin?', 'How many people live in Berlin?'], ['Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.'],  padding=True, truncation=True, return_tensors="pt")
model.eval() # turns off dropout
with torch.no_grad(): #save memory and speed by not calculating backward pass or storing gradient
    scores = model(**features).logits
    print(scores)


Using this same approach you will implement a reranking function that given a set of candidate documents and a query, will rerank them with a cross encoder. 